In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objs as go
import plotly
import os
import seaborn as sns
from plotly.graph_objs import *
from ipywidgets import Image, HTML, Layout, Output, VBox, HBox
from IPython.display import Audio, display

In [2]:
import io
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [3]:
df = pd.read_pickle('spec_df.pkl')

In [4]:
df

level_0  index     Name     Start  Duration Time Format Type  \
0        3335     21  chitter  0:35.655  0:00.044     decimal  Cue   
1        7515     39  chitter  0:34.365  0:00.049     decimal  Cue   
2        2894     25  chitter  0:45.650  0:00.017     decimal  Cue   
3        3810    168  chitter  1:33.687  0:00.054     decimal  Cue   
4         816     44  chitter  0:31.220  0:00.048     decimal  Cue   
...       ...    ...      ...       ...       ...         ...  ...   
1376     7602    126  chirpgr  1:20.869  0:00.081     decimal  Cue   
1377     7603    127  chirpgr  1:21.076  0:00.085     decimal  Cue   
1378     7604    128  chirpgr  1:21.286  0:00.055     decimal  Cue   
1379     7605    129  chirpgr  1:21.440  0:00.092     decimal  Cue   
1380     7606    130  chirpgr  1:21.753  0:00.063     decimal  Cue   

     Description  start_s  duration_s  ...  \
0            NaN   35.655       0.044  ...   
1            NaN   34.365       0.049  ...   
2            NaN   45.650       0.017  ...   
3            NaN   93.687       0.054  ...   
4            NaN   31.220       0.048  ...   
...          ...      ...         ...  ...   
1376         NaN   80.869       0.081  ...   
1377         NaN   81.076       0.085  ...   
1378         NaN   81.286       0.055  ...   
1379         NaN   81.440       0.092  ...   
1380         NaN   81.753       0.063  ...   

                                              raw_audio  \
0     [0.01991272, 0.017089844, 0.012939453, 0.00761...   
1     [0.0040740967, 0.003753662, 0.004348755, 0.004...   
2     [0.0064849854, 0.0030822754, 3.0517578e-05, -0...   
3     [0.020324707, 0.019714355, 0.024383545, 0.0114...   
4     [-0.0284729, -0.029174805, -0.027862549, -0.02...   
...                                                 ...   
1376  [0.011260986, 0.008590698, 0.004425049, 0.0021...   
1377  [0.009857178, 0.010314941, 0.0051574707, -0.00...   
1378  [-0.014831543, -0.01335144, -0.010055542, -0.0...   
1379  [0.002319336, -0.00074768066, -0.0029754639, -...   
1380  [-0.011932373, -0.0029449463, 0.0039367676, 0....   

                   project_id   labels    label  \
0             7.3.2004.PQ.d3b  chitter  chitter   
1              6.28.2003.PS.5  chitter  chitter   
2             9.15.2003.PQ.20  chitter  chitter   
3             6.27.2004.PS.d3  chitter  chitter   
4     6.3.2004.PQj.d7aconinue  chitter  chitter   
...                       ...      ...      ...   
1376           6.28.2003.PS.5  chirpgr  chirpgr   
1377           6.28.2003.PS.5  chirpgr  chirpgr   
1378           6.28.2003.PS.5  chirpgr  chirpgr   
1379           6.28.2003.PS.5  chirpgr  chirpgr   
1380           6.28.2003.PS.5  chirpgr  chirpgr   

                                           spectrograms  \
0     [[72.63461, 76.67954, 80.36285, 82.02532, 82.0...   
1     [[72.26875, 71.64608, 69.91204, 68.42412, 68.8...   
2     [[59.360443, 58.412537, 56.778717, 56.808647, ...   
3     [[58.96704, 58.52694, 57.505547, 56.508327, 55...   
4     [[86.30569, 85.66044, 83.63323, 80.28586, 76.9...   
...                                                 ...   
1376  [[72.91542, 73.506004, 74.72987, 76.08979, 76....   
1377  [[73.59757, 72.83039, 70.475525, 66.8248, 66.0...   
1378  [[62.707687, 64.40575, 67.55345, 69.82164, 70....   
1379  [[69.94864, 68.96461, 66.08038, 63.0766, 63.18...   
1380  [[70.096924, 70.675804, 72.70071, 75.25197, 76...   

                                  denoised_spectrograms     UMAP1      UMAP2  \
0     [[3.485077, 6.833214, 10.330238, 12.100716, 13...  3.359666  10.503068   
1     [[11.882889, 9.145069, 6.9758453, 6.4505577, 6...  1.329389   7.079291   
2     [[18.019005, 16.870316, 15.179089, 16.113388, ...  0.041436   9.702542   
3     [[-7.927292, -8.696785, -9.727798, -9.092411, ...  1.798064   7.024569   
4     [[20.779015, 19.060211, 15.956535, 11.601753, ...  2.273248   7.772343   
...                                                 ...       ...        ...   
1376  [[9.3647995, 9.

In [5]:
SR = 48000
FFT_WIN = 0.03 # FFT_WIN*samplerate = length of fft/n_fft (number of audio frames that go in one fft)
FFT_HOP = FFT_WIN/8 # FFT_HOP*samplerate = n of audio frames between successive ffts
FMAX = SR/2

In [8]:
# fmin and fmax are different for preprocessed specs!
librosa.mel_frequencies(n_mels=40, fmin=0.0, fmax=FMAX, htk=False)

array([    0.        ,   104.65819224,   209.31638447,   313.97457671,
         418.63276894,   523.29096118,   627.94915342,   732.60734565,
         837.26553789,   941.92373012,  1049.21141573,  1168.79160686,
        1302.00053085,  1450.39147474,  1615.694756  ,  1799.83789895,
        2004.96811077,  2233.477319  ,  2488.03006278,  2771.59456271,
        3087.47733196,  3439.36173191,  3831.35092214,  4268.01570548,
        4754.44782596,  5296.31934126,  5899.94876196,  6572.37472873,
        7321.43808661,  8155.87331345,  9085.41036859, 10120.88814935,
       11274.38087835, 12559.3388954 , 13990.74549561, 15585.29164258,
       17361.57059398, 19340.29470877, 21544.5369644 , 24000.        ])

In [6]:
FMIN = 620
FMAX = 14000

In [12]:
# preprocessed specs

image_data = {}

for i,dat in enumerate(df.denoised_spectrograms):
    print('\rProcessing i:',i, end='')
    dat = np.asarray(df.iloc[i]['preprocessed_specs'])       
    plt.figure()
    librosa.display.specshow(dat,sr=SR, hop_length=int(FFT_HOP * SR) , fmin=FMIN, fmax=FMAX, y_axis='mel', x_axis='s',cmap='inferno')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    byte_im = buf.getvalue()
    image_data[df.iloc[i]['callID']] = byte_im
    plt.close()

# Store data (serialize)
with open('image_data.pickle', 'wb') as handle:
    pickle.dump(image_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# original specs

image_data = {}

for i,dat in enumerate(df.denoised_spectrograms):
    print('\rProcessing i:',i, end='')
    #dat = np.asarray(df.iloc[i]['preprocessed_specs'])    
    dat = np.asarray(df.iloc[i]['denoised_spectrograms'])    
    plt.figure()
    librosa.display.specshow(dat,sr=SR, hop_length=int(FFT_HOP * SR) , fmin=FMIN, fmax=FMAX, y_axis='mel', x_axis='s',cmap='inferno')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    byte_im = buf.getvalue()
    image_data[df.iloc[i]['callID']] = byte_im
    plt.close()

# Store data (serialize)
with open('image_data_ori.pickle', 'wb') as handle:
    pickle.dump(image_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

Processing i: 1

/Users/marathomas/opt/anaconda3/envs/coati_feeding_env/lib/python3.7/site-packages/librosa/display.py:862: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/Users/marathomas/opt/anaconda3/envs/coati_feeding_env/lib/python3.7/site-packages/librosa/display.py:862: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)


Processing i: 1321

In [9]:
%pwd

'/Users/marathomas/Documents/MPI_work/coati_feeding'